In [42]:
from __future__ import print_function

import numpy as np

import scipy.misc
import scipy.io

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.applications import vgg19
from keras.models import load_model, Model, Sequential
from keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Lambda, Flatten, Dense
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.core import Reshape
from keras.layers.merge import concatenate
import tensorflow as tf

# Set backend as tensorflow
from keras import backend as K
K.set_image_dim_ordering("tf")

from PIL import Image

import os

%matplotlib inline

In [43]:
import h5py

weights = h5py.File("yolo.h5", "r")

In [44]:
BASE_DIR = "./ships-in-satellite-imagery/shipsnet/"
TRAIN_ROWS = .2
IMAGE_HEIGHT = 80
IMAGE_WIDTH = 80
COLOR_CHANNELS = 3

In [45]:
file_list = os.listdir(BASE_DIR)
total_images = len(file_list)

x_train_rows = int(total_images * TRAIN_ROWS)
x_test_rows = total_images - x_train_rows

x_train = np.zeros((x_train_rows, IMAGE_HEIGHT, IMAGE_WIDTH, COLOR_CHANNELS))
y_train = np.zeros((x_train_rows, 1))
x_test = np.zeros((x_test_rows, IMAGE_HEIGHT, IMAGE_WIDTH, COLOR_CHANNELS))
y_test = np.zeros((x_test_rows, 1))

i = 0
for f in random.:
    x_train[i] = plt.imread(BASE_DIR + f, "r")
    y_train[i] = int(f.split("_", 1)[0])
    i += 1

i = 0
for f in file_list[x_train_rows:]:
    x_test[i] = plt.imread(BASE_DIR + f, "r")
    y_test[i] = int(f.split("_", 1)[0])
    i += 1

SyntaxError: invalid syntax (<ipython-input-45-acf4ed73ce0f>, line 13)

In [46]:
input_image = Input(shape=(80, 80, 3), name="input_1")

x = Conv2D(32, (3, 3), name="conv_1", padding="same")(input_image)
x = BatchNormalization(name='norm_1')(x)
x = LeakyReLU(alpha=0.1, name="leaky_relu_1")(x)
x = MaxPooling2D(pool_size=(2, 2), name="max_pool_1")(x)
x = Flatten(name="flatten_1")(x)
x = Dense(1, name="dense_1")(x)

model = Model(input_image, x)

# model.summary()

In [14]:
# For a binary classification problem
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [15]:
# Train the model, iterating on the data in batches of 32 samples
model.fit(x_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
560/560 [==============================] - 12s 21ms/step - loss: 0.0386 - acc: 0.0339
Epoch 2/10
560/560 [==============================] - 10s 19ms/step - loss: 1.0000e-07 - acc: 0.0000e+00
Epoch 3/10
384/560 [===================>..........] - ETA: 3s - loss: 1.0000e-07 - acc: 0.0000e+00

KeyboardInterrupt: 

In [51]:
a = np.random.shuffle([5, 10, 5, 10])

print(a)

None
